In [5]:
import numpy as np
import pandas as pd

In [6]:
df_movies = pd.read_csv("../../Datasets/IMDb_scraper.csv")
df_movies.head()

,web-scraper-order,web-scraper-start-url,movie_links,movie_links-href,rating,director_0-1,director_0-2,director_0-3,writers_0-1,writers_0-2,...,stars_0-3,stars_1-1,stars_1-2,stars_1-3,writers_1-1,writers_1-2,writers_1-3,director_1-1,director_1-2,director_1-3
0,1762884594-1,https://www.imdb.com/chart/top/?ref_=hm_nv_menu,Lost Ladies,https://www.imdb.com/title/tt21626284/?ref_=ch...,8.3,Kiran Rao,NaN,NaN,Biplab Goswami,Sneha Desai,...,Sparsh Shrivastava,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1762884596-2,https://www.imdb.com/chart/top/?ref_=hm_nv_menu,Drishyam,https://www.imdb.com/title/tt4430212/?ref_=cht...,8.2,Nishikant Kamat,NaN,NaN,Jeethu Joseph,Upendra Sidhaye,...,Tabu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1762884600-3,https://www.imdb.com/chart/top/?ref_=hm_nv_menu,Gangs of Wasseypur,https://www.imdb.com/title/tt1954470/?ref_=cht...,8.2,Anurag Kashyap,NaN,NaN,Akhilesh Jaiswal,Anurag Kashyap,...,Tigmanshu Dhulia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1762884604-4,https://www.imdb.com/chart/top/?ref_=hm_nv_menu,To Be or Not to Be,https://www.imdb.com/title/tt0035446/?ref_=cht...,8.1,NaN,NaN,NaN,NaN,NaN,...,NaN,Carole Lombard,Jack Benny,Robert Stack,Melchior Lengyel,Edwin Justus Mayer,Ernst Lubitsch,Ernst Lubitsch,NaN,NaN
4,1762884608-5,https://www.imdb.com/chart/top/?ref_=hm_nv_menu,Demon Slayer: Kimetsu no Yaiba - The Movie: Mu...,https://www.imdb.com/title/tt11032374/?ref_=ch...,8.2,Haruo Sotozaki,NaN,NaN,Koyoharu Gotouge,NaN,...,Hiro Shimono,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def replace_empty(
    df: pd.DataFrame,
    from_: str = "director_1",
    to_: int = "director_0",
    start: int = 1,
    end: int = 3,
    treat_blank_as_na: bool = True,
    drop_backup: bool = True
) -> pd.DataFrame:
    
    df = df.copy()

   
    primary_cols = [f"{to_}-{i}" for i in range(start, end + 1)]
    backup_cols  = [f"{from_}-{i}"  for i in range(start, end + 1)]

    primary_cols = [c for c in primary_cols if c in df.columns]
    backup_cols  = [c for c in backup_cols  if c in df.columns]

    if not primary_cols or not backup_cols or (len(primary_cols) != len(backup_cols)):
        print("cols not foutd")
        return df


    anchor_col = f"{to_}-{start}"
    if anchor_col not in df.columns:
        anchor_col = primary_cols[0]

    if treat_blank_as_na:
        anchor_values = df[anchor_col].replace(r'^\s*$', np.nan, regex=True)
    else:
        anchor_values = df[anchor_col]

    mask = anchor_values.isna()

    common_len = min(len(primary_cols), len(backup_cols))
    pcols = primary_cols[:common_len]
    bcols = backup_cols[:common_len]


    df.loc[mask, pcols] = df.loc[mask, bcols].values

  
    if drop_backup and bcols:
        df = df.drop(columns=bcols, errors="ignore")

    return df
    

In [8]:
df_movies = replace_empty(df_movies)
df_movies = replace_empty(df_movies,from_="writers_1",to_="writers_0")
df_movies = replace_empty(df_movies,from_="stars_1",to_="stars_0")

df_movies.head()

,web-scraper-order,web-scraper-start-url,movie_links,movie_links-href,rating,director_0-1,director_0-2,director_0-3,writers_0-1,writers_0-2,writers_0-3,stars_0-1,stars_0-2,stars_0-3
0,1762884594-1,https://www.imdb.com/chart/top/?ref_=hm_nv_menu,Lost Ladies,https://www.imdb.com/title/tt21626284/?ref_=ch...,8.3,Kiran Rao,NaN,NaN,Biplab Goswami,Sneha Desai,Divy Nidhi Sharma,Nitanshi Goel,Pratibha Ranta,Sparsh Shrivastava
1,1762884596-2,https://www.imdb.com/chart/top/?ref_=hm_nv_menu,Drishyam,https://www.imdb.com/title/tt4430212/?ref_=cht...,8.2,Nishikant Kamat,NaN,NaN,Jeethu Joseph,Upendra Sidhaye,NaN,Ajay Devgn,Shriya Saran,Tabu
2,1762884600-3,https://www.imdb.com/chart/top/?ref_=hm_nv_menu,Gangs of Wasseypur,https://www.imdb.com/title/tt1954470/?ref_=cht...,8.2,Anurag Kashyap,NaN,NaN,Akhilesh Jaiswal,Anurag Kashyap,Sachin K. Ladia,Manoj Bajpayee,Nawazuddin Siddiqui,Tigmanshu Dhulia
3,1762884604-4,https://www.imdb.com/chart/top/?ref_=hm_nv_menu,To Be or Not to Be,https://www.imdb.com/title/tt0035446/?ref_=cht...,8.1,Ernst Lubitsch,NaN,NaN,Melchior Lengyel,Edwin Justus Mayer,Ernst Lubitsch,Carole Lombard,Jack Benny,Robert Stack
4,1762884608-5,https://www.imdb.com/chart/top/?ref_=hm_nv_menu,Demon Slayer: Kimetsu no Yaiba - The Movie: Mu...,https://www.imdb.com/title/tt11032374/?ref_=ch...,8.2,Haruo Sotozaki,NaN,NaN,Koyoharu Gotouge,NaN,NaN,Natsuki Hanae,Akari Kitô,Hiro Shimono


In [9]:
df_movies.to_csv("IMDb_scraper_cleaned.csv", index=False)